# University System Model

This model represents the various processes and entities involved in managing course registrations at a university. Below is a detailed explanation of each component and the consequences of the decisions that went into the development of this model.

Before we begin, we need to reference the Jinaga packages.

In [ ]:
#r "nuget: Jinaga, 1.1.15"
#r "nuget: Jinaga.UnitTest, 1.1.2"
#r "nuget: Jinaga.Notebooks, 1.1.3"

Installed Packages Jinaga, 1.1.15 Jinaga.Notebooks, 1.1.3 Jinaga.UnitTest, 1.1.2

In [ ]:
using Jinaga;
using Jinaga.Notebooks;
using Jinaga.UnitTest;
using Jinaga.Extensions;

## Facts

The model is made of historical facts. Each fact is a record of a person, entity, or decision that pertains to the system.

### Application and Enrollment

- **Student**: Represents each student at the university. Each student has a unique identifier (public key).
- **Organization**: Represents the university itself or any other educational institution within the system.
- **Application**: When a student applies to the university, an application record is created. This links the student to the organization and includes the date of application.
- **Enrollment**: If the application is accepted, an enrollment record is created, indicating that the student is now enrolled in the university.

In [ ]:
[FactType("University.Student")]
public record Student(string publicKey);

[FactType("University.Organization")]
public record Organization(string identifier);

[FactType("University.Application")]
public record Application(Student student, Organization organization, DateTime appliedAt);

[FactType("University.Enrollment")]
public record Enrollment(Application application);

[FactType("University.Rejection")]
public record Rejection(Application application);

Renderer.RenderTypes(typeof(Application), typeof(Enrollment), typeof(Rejection))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- University.Application -->
 
 University.Application 
 
 University.Application 
 
<!-- University.Student -->
 
 University.Student 
 
 University.Student 
 
<!-- University.Application->University.Student -->
 
 University.Application->University.Student 
 
 
 student 
 
<!-- University.Organization -->
 
 University.Organization 
 
 University.Organization 
 
<!-- University.Application->University.Organization -->
 
 University.Application->University.Organization 
 
 
 organization 
 
<!-- University.Enrollment -->
 
 University.Enrollment 
 
 University.Enrollment 
 
<!-- University.Enrollment->University.Application -->
 
 University.Enrollment->University.Application 
 
 
 application 
 
<!-- University.Rejection -->
 
 University.Rejection 
 
 University.Rejection 
 
<!-- University.Rejection->University.Application -->
 
 University.Rejection->University.Application 
 
 
 application

### Curriculum

- **Course**: Represents the different courses offered by the university, each with a unique code and name.
- **Semester**: Represents the academic terms (e.g., Spring 2022, Fall 2022) during which courses are offered.

In [ ]:
[FactType("University.Course")]
public record Course(Organization organization, string code, string name);

[FactType("University.Semester")]
public record Semester(Organization organization, int year, string term);

Renderer.RenderTypes(typeof(Course), typeof(Semester))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- University.Course -->
 
 University.Course 
 
 University.Course 
 
<!-- University.Organization -->
 
 University.Organization 
 
 University.Organization 
 
<!-- University.Course->University.Organization -->
 
 University.Course->University.Organization 
 
 
 organization 
 
<!-- University.Semester -->
 
 University.Semester 
 
 University.Semester 
 
<!-- University.Semester->University.Organization -->
 
 University.Semester->University.Organization 
 
 
 organization

### Instruction

- **Instructor**: Represents the faculty members who teach the courses.
- **Offering**: Represents a specific instance of a course being offered in a particular semester, taught by a specific instructor. It includes details like the days of the week and the time the course is held.
- **Offering Location**: A change to the location of the offering. Each new change refers to the one that it replaces.
- **Offering Delete**: The deletion of an offering from the course catalog.

In [ ]:
[FactType("University.Instructor")]
public record Instructor(Organization organization, string name);

[FactType("University.Offering")]
public record Offering(Course course, Semester semester, Instructor instructor, string days, string time)
{
    public Relation<OfferingLocation> Locations => Relation.Define<OfferingLocation>(() =>
        from location in this.Successors().OfType<OfferingLocation>(location => location.offering)
        where location.Successors().No<OfferingLocation>(next => next.prior)
        select location
    );
}

[FactType("University.Offering.Location")]
public record OfferingLocation(Offering offering, string building, string room, OfferingLocation[] prior);

[FactType("University.Offering.Delete")]
public record OfferingDelete(Offering offering, DateTime deletedAt);

Renderer.RenderTypes(typeof(Instructor), typeof(Offering), typeof(OfferingLocation), typeof(OfferingDelete))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- University.Instructor -->
 
 University.Instructor 
 
 University.Instructor 
 
<!-- University.Organization -->
 
 University.Organization 
 
 University.Organization 
 
<!-- University.Instructor->University.Organization -->
 
 University.Instructor->University.Organization 
 
 
 organization 
 
<!-- University.Offering -->
 
 University.Offering 
 
 University.Offering 
 
<!-- University.Offering->University.Instructor -->
 
 University.Offering->University.Instructor 
 
 
 instructor 
 
<!-- University.Course -->
 
 University.Course 
 
 University.Course 
 
<!-- University.Offering->University.Course -->
 
 University.Offering->University.Course 
 
 
 course 
 
<!-- University.Semester -->
 
 University.Semester 
 
 University.Semester 
 
<!-- University.Offering->University.Semester -->
 
 University.Offering->University.Semester 
 
 
 semester 
 
<!-- University.Course->University.Organization -->
 
 University.Course->University.Organization 
 
 
 organization 
 
<!-- University.Semester->University.Organization -->
 
 University.Semester->University.Organization 
 
 
 organization 
 
<!-- University.Offering.Location -->
 
 University.Offering.Location 
 
 University.Offering.Location 
 
<!-- University.Offering.Location->University.Offering -->
 
 University.Offering.Location->University.Offering 
 
 
 offering 
 
<!-- University.Offering.Location->University.Offering.Location -->
 
 University.Offering.Location->University.Offering.Location 
 
 
 prior 
 
<!-- University.Offering.Delete -->
 
 University.Offering.Delete 
 
 University.Offering.Delete 
 
<!-- University.Offering.Delete->University.Offering -->
 
 University.Offering.Delete->University.Offering 
 
 
 offering

### Registrations and Outcomes

- **Registration**: When a student enrolls in a course offering, a registration record is created. This links the student's enrollment to the specific course offering.
- **Drop**: If a student decides to drop a course, a drop record is created.
- **Fail**: If a student fails a course, a fail record is created with the grade.
- **Complete**: If a student successfully completes a course, a complete record is created with the grade.

In [ ]:
[FactType("University.Registration")]
public record Registration(Enrollment enrollment, Offering offering);

[FactType("University.Drop")]
public record Drop(Registration registration);

[FactType("University.Fail")]
public record Fail(Registration registration, int grade);

[FactType("University.Complete")]
public record Complete(Registration registration, int grade);

Renderer.RenderTypes(typeof(Registration), typeof(OfferingLocation), typeof(Drop), typeof(Fail), typeof(Complete))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- University.Registration -->
 
 University.Registration 
 
 University.Registration 
 
<!-- University.Enrollment -->
 
 University.Enrollment 
 
 University.Enrollment 
 
<!-- University.Registration->University.Enrollment -->
 
 University.Registration->University.Enrollment 
 
 
 enrollment 
 
<!-- University.Offering -->
 
 University.Offering 
 
 University.Offering 
 
<!-- University.Registration->University.Offering -->
 
 University.Registration->University.Offering 
 
 
 offering 
 
<!-- University.Application -->
 
 University.Application 
 
 University.Application 
 
<!-- University.Enrollment->University.Application -->
 
 University.Enrollment->University.Application 
 
 
 application 
 
<!-- University.Course -->
 
 University.Course 
 
 University.Course 
 
<!-- University.Offering->University.Course -->
 
 University.Offering->University.Course 
 
 
 course 
 
<!-- University.Semester -->
 
 University.Semester 
 
 University.Semester 
 
<!-- University.Offering->University.Semester -->
 
 University.Offering->University.Semester 
 
 
 semester 
 
<!-- University.Instructor -->
 
 University.Instructor 
 
 University.Instructor 
 
<!-- University.Offering->University.Instructor -->
 
 University.Offering->University.Instructor 
 
 
 instructor 
 
<!-- University.Offering.Location -->
 
 University.Offering.Location 
 
 University.Offering.Location 
 
<!-- University.Offering.Location->University.Offering -->
 
 University.Offering.Location->University.Offering 
 
 
 offering 
 
<!-- University.Offering.Location->University.Offering.Location -->
 
 University.Offering.Location->University.Offering.Location 
 
 
 prior 
 
<!-- University.Drop -->
 
 University.Drop 
 
 University.Drop 
 
<!-- University.Drop->University.Registration -->
 
 University.Drop->University.Registration 
 
 
 registration 
 
<!-- University.Fail -->
 
 University.Fail 
 
 University.Fail 
 
<!-- University.Fail->University.Registration -->
 
 University.Fail->University.Registration 
 
 
 registration 
 
<!-- University.Complete -->
 
 University.Complete 
 
 University.Complete 
 
<!-- University.Complete->University.Registration -->
 
 University.Complete->University.Registration 
 
 
 registration 
 
<!-- University.Student -->
 
 University.Student 
 
 University.Student 
 
<!-- University.Application->University.Student -->
 
 University.Application->University.Student 
 
 
 student 
 
<!-- University.Organization -->
 
 University.Organization 
 
 University.Organization 
 
<!-- University.Application->University.Organization -->
 
 University.Application->University.Organization 
 
 
 organization 
 
<!-- University.Course->University.Organization -->
 
 University.Course->University.Organization 
 
 
 organization 
 
<!-- University.Semester->University.Organization -->
 
 University.Semester->University.Organization 
 
 
 organization 
 
<!-- University.Instructor->University.Organization -->
 
 University.Instructor->University.Organization 
 
 
 organization

## Practical Example

Let's construct some facts to illustrate the development of curriculum by the university. We will initialize a student fact, and populate a set of courses.

In [ ]:
var j = JinagaTest.Create();

var university = await j.Fact(new Organization("6003"));
var student = await j.Fact(new Student("---PUBLIC KEY---"));
var application = await j.Fact(new Application(student, university, DateTime.Parse("2022-02-04")));
var enrollment = await j.Fact(new Enrollment(application));

List<Course> courses = [
    await j.Fact(new Course(university, "CS 101", "Introduction to Computer Science")),
    await j.Fact(new Course(university, "CS 201", "Data Structures and Algorithms")),
    await j.Fact(new Course(university, "CS 301", "Software Engineering")),
    await j.Fact(new Course(university, "CS 401", "Artificial Intelligence")),
    await j.Fact(new Course(university, "CS 501", "Machine Learning")),
    await j.Fact(new Course(university, "CS 601", "Quantum Computing"))
];

Next, we will staff the university with instructors and create course offerings. Based on this history, we can generate a course catalog for the upcoming semester.

In [ ]:
List<Instructor> instructors = [
    await j.Fact(new Instructor(university, "Dr. Smith")),
    await j.Fact(new Instructor(university, "Dr. Jones")),
    await j.Fact(new Instructor(university, "Dr. Lee")),
    await j.Fact(new Instructor(university, "Dr. Kim")),
    await j.Fact(new Instructor(university, "Dr. Patel")),
    await j.Fact(new Instructor(university, "Dr. Singh"))
];

List<Semester> semesters = [
    await j.Fact(new Semester(university, 2022, "Spring")),
    await j.Fact(new Semester(university, 2022, "Summer")),
    await j.Fact(new Semester(university, 2022, "Fall")),
    await j.Fact(new Semester(university, 2023, "Spring")),
    await j.Fact(new Semester(university, 2023, "Summer")),
    await j.Fact(new Semester(university, 2023, "Fall"))
];

var random = new Random(29693);

List<Offering> offerings = new List<Offering>();
string[] possibleDays = new string[] { "MF", "TTr", "MW", "WF" };
string[] possibleBuildings = new string[] { "Building A", "Building B", "Building C", "Building D" };
string[] possibleRooms = new string[] { "101", "102", "103", "104" };
for (int i = 0; i < 100; i++)
{
    var course = courses[random.Next(courses.Count)];
    var semester = semesters[random.Next(semesters.Count)];
    var instructor = instructors[random.Next(instructors.Count)];
    var days = possibleDays[random.Next(possibleDays.Length)];
    var time = (8 + random.Next(12)).ToString() + ":00";
    var building = possibleBuildings[random.Next(possibleBuildings.Length)];
    var room = possibleRooms[random.Next(possibleRooms.Length)];
    var offering = await j.Fact(new Offering(course, semester, instructor, days, time));
    var location = await j.Fact(new OfferingLocation(offering, building, room, new OfferingLocation[0]));
    offerings.Add(offering);
}

### Specification

We now have enough information to generate a projection. Let's define a specification that gives us a list of courses offered in the upcoming semester.

In [ ]:
// List offerings for the current semester
var offeringsForSemester = Given<Semester>.Match(semester =>
    from offering in semester.Successors().OfType<Offering>(o => o.semester)
    from course in offering.course.Successors().OfType<Course>(c => c)
    from instructor in offering.instructor.Successors().OfType<Instructor>(i => i)
    select new
    {
        CourseCode = course.code,
        CourseName = course.name,
        Days = offering.days,
        Time = offering.time,
        Instructor = instructor.name,
        Locations =
            from location in offering.Locations
            select new
            {
                Building = location.building,
                Room = location.room
            }
    });
var currentSemester = semesters[1];
var offeringsThisSemester = await j.Query(offeringsForSemester, currentSemester);

offeringsThisSemester
    .Select(o => new {
        o.CourseCode,
        o.CourseName,
        o.Days,
        o.Time,
        o.Instructor,
        Locations = o.Locations.Select(l => l.Building + " " + l.Room).FirstOrDefault()
    })
    .AsTable()

index,CourseCode,CourseName,Days,Time,Instructor,Locations
0,CS 101,Introduction to Computer Science,TTr,10:00,Dr. Patel,Building A 101
1,CS 401,Artificial Intelligence,WF,14:00,Dr. Smith,Building D 103
2,CS 401,Artificial Intelligence,TTr,16:00,Dr. Smith,Building C 101
3,CS 101,Introduction to Computer Science,TTr,13:00,Dr. Jones,Building B 104
4,CS 301,Software Engineering,WF,19:00,Dr. Jones,Building B 102
5,CS 101,Introduction to Computer Science,MF,14:00,Dr. Kim,Building A 102
6,CS 201,Data Structures and Algorithms,TTr,14:00,Dr. Smith,Building A 104
7,CS 601,Quantum Computing,MF,17:00,Dr. Singh,Building B 104
8,CS 201,Data Structures and Algorithms,TTr,10:00,Dr. Patel,Building B 104
9,CS 501,Machine Learning,MW,11:00,Dr. Lee,Building B 101


The data behind each row of this projection are interrelated facts. Each offering brings together a course, a semester, an instructor, and a location.

For example, Dr. Patel and Dr. Jones are both teaching the "Introduction to Computer Science" course in the Spring 2022 semester. The historical model shows two offerings of the same course in the same semester, each taught by a different instructor.

In [ ]:
j.RenderFacts(offerings.Where(o => o.semester == currentSemester && o.course.code == "CS 101").Take(2))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== -->
 
 6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== 
 
 University.Organization 
 
 identifier 
 
 6003 
 
<!-- YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg== -->
 
 YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg== 
 
 University.Course 
 
 code 
 
 CS 101 
 
 name 
 
 Introduction to Comp... 
 
<!-- YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== -->
 
 YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== 
 
 
 organization 
 
<!-- cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg== -->
 
 cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg== 
 
 University.Semester 
 
 year 
 
 2022 
 
 term 
 
 Summer 
 
<!-- cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== -->
 
 cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== 
 
 
 organization 
 
<!-- h7wbdX6KTqXpukbJbbAf2gDj36IqDtjx8U7CoI436rbFwqGS9k3Mse/Wm5MyChWb3Dwvuic+6VNFmYaLhbvM7w== -->
 
 h7wbdX6KTqXpukbJbbAf2gDj36IqDtjx8U7CoI436rbFwqGS9k3Mse/Wm5MyChWb3Dwvuic+6VNFmYaLhbvM7w== 
 
 University.Instructor 
 
 name 
 
 Dr. Patel 
 
<!-- h7wbdX6KTqXpukbJbbAf2gDj36IqDtjx8U7CoI436rbFwqGS9k3Mse/Wm5MyChWb3Dwvuic+6VNFmYaLhbvM7w==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== -->
 
 h7wbdX6KTqXpukbJbbAf2gDj36IqDtjx8U7CoI436rbFwqGS9k3Mse/Wm5MyChWb3Dwvuic+6VNFmYaLhbvM7w==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== 
 
 
 organization 
 
<!-- gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ== -->
 
 gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ== 
 
 University.Offering 
 
 days 
 
 TTr 
 
 time 
 
 10:00 
 
 
<!-- gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ==->YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg== -->
 
 gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ==->YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg== 
 
 
 course 
 
<!-- gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ==->cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg== -->
 
 gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ==->cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg== 
 
 
 semester 
 
<!-- gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ==->h7wbdX6KTqXpukbJbbAf2gDj36IqDtjx8U7CoI436rbFwqGS9k3Mse/Wm5MyChWb3Dwvuic+6VNFmYaLhbvM7w== -->
 
 gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ==->h7wbdX6KTqXpukbJbbAf2gDj36IqDtjx8U7CoI436rbFwqGS9k3Mse/Wm5MyChWb3Dwvuic+6VNFmYaLhbvM7w== 
 
 
 instructor 
 
<!-- DHIqBfao652bzZNeIxWi5aq9Wn8ZWzOki9O5x/JaHFQm79FBX4L/CrtIm+f/2tOC6Ar5o0hgz1dURYm36PQ2OQ== -->
 
 DHIqBfao652bzZNeIxWi5aq9Wn8ZWzOki9O5x/JaHFQm79FB

### Student Enrollment and Registration

Imagine a student named Alice applies to the university. Her application is accepted, and she enrolls in the university. She then registers for a course called "Introduction to Computer Science" taught by Dr. Patel in the Summer 2022 semester. Alice completes the course with a grade of 93.

In [ ]:
var registration = await j.Fact(new Registration(enrollment, offerings[0]));
var complete = await j.Fact(new Complete(registration, 93));
j.RenderFacts(complete)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- Y62VsMJeGGPdhDKlSk7sw2ntmsSiQSZZUpXb+xnboJ4PFrrhNmI3d9TECCCJGIGM2gbbNbJgMovr5DDXKm1ypA== -->
 
 Y62VsMJeGGPdhDKlSk7sw2ntmsSiQSZZUpXb+xnboJ4PFrrhNmI3d9TECCCJGIGM2gbbNbJgMovr5DDXKm1ypA== 
 
 University.Student 
 
 publicKey 
 
 ---PUBLIC KEY--- 
 
<!-- 6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== -->
 
 6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== 
 
 University.Organization 
 
 identifier 
 
 6003 
 
<!-- wOqS3J8VTD9rpSiA/xNm3dlqqx7lEs0Pq4Wpa7CtdE2b+LAznXrBDfI0N0+c2PzCrb7SPga6+M1pg6IXBCGmSA== -->
 
 wOqS3J8VTD9rpSiA/xNm3dlqqx7lEs0Pq4Wpa7CtdE2b+LAznXrBDfI0N0+c2PzCrb7SPga6+M1pg6IXBCGmSA== 
 
 University.Application 
 
 appliedAt 
 
 2022-02-04T06:00:00.... 
 
<!-- wOqS3J8VTD9rpSiA/xNm3dlqqx7lEs0Pq4Wpa7CtdE2b+LAznXrBDfI0N0+c2PzCrb7SPga6+M1pg6IXBCGmSA==->Y62VsMJeGGPdhDKlSk7sw2ntmsSiQSZZUpXb+xnboJ4PFrrhNmI3d9TECCCJGIGM2gbbNbJgMovr5DDXKm1ypA== -->
 
 wOqS3J8VTD9rpSiA/xNm3dlqqx7lEs0Pq4Wpa7CtdE2b+LAznXrBDfI0N0+c2PzCrb7SPga6+M1pg6IXBCGmSA==->Y62VsMJeGGPdhDKlSk7sw2ntmsSiQSZZUpXb+xnboJ4PFrrhNmI3d9TECCCJGIGM2gbbNbJgMovr5DDXKm1ypA== 
 
 
 student 
 
<!-- wOqS3J8VTD9rpSiA/xNm3dlqqx7lEs0Pq4Wpa7CtdE2b+LAznXrBDfI0N0+c2PzCrb7SPga6+M1pg6IXBCGmSA==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== -->
 
 wOqS3J8VTD9rpSiA/xNm3dlqqx7lEs0Pq4Wpa7CtdE2b+LAznXrBDfI0N0+c2PzCrb7SPga6+M1pg6IXBCGmSA==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== 
 
 
 organization 
 
<!-- XjV795X6ORSBepQ19wRqQfaazlr23uSmE8yDhtEndxZi7iJwmOnzLTJUnDFcG8fzDVnf6i05clQMFymWCu6aMA== -->
 
 XjV795X6ORSBepQ19wRqQfaazlr23uSmE8yDhtEndxZi7iJwmOnzLTJUnDFcG8fzDVnf6i05clQMFymWCu6aMA== 
 
 University.Enrollment 
 
<!-- XjV795X6ORSBepQ19wRqQfaazlr23uSmE8yDhtEndxZi7iJwmOnzLTJUnDFcG8fzDVnf6i05clQMFymWCu6aMA==->wOqS3J8VTD9rpSiA/xNm3dlqqx7lEs0Pq4Wpa7CtdE2b+LAznXrBDfI0N0+c2PzCrb7SPga6+M1pg6IXBCGmSA== -->
 
 XjV795X6ORSBepQ19wRqQfaazlr23uSmE8yDhtEndxZi7iJwmOnzLTJUnDFcG8fzDVnf6i05clQMFymWCu6aMA==->wOqS3J8VTD9rpSiA/xNm3dlqqx7lEs0Pq4Wpa7CtdE2b+LAznXrBDfI0N0+c2PzCrb7SPga6+M1pg6IXBCGmSA== 
 
 
 application 
 
<!-- YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg== -->
 
 YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg== 
 
 University.Course 
 
 code 
 
 CS 101 
 
 name 
 
 Introduction to Comp... 
 
<!-- YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== -->
 
 YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== 
 
 
 organization 
 
<!-- cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg== -->
 
 cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg== 
 
 University.Semester 
 
 year 
 
 2022 
 
 term 
 
 Summer 
 
<!-- cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== -->
 
 cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== 
 
 
 organization 
 
<!-- h7wbdX6KTqXpukbJbbAf2gDj36IqDtjx8U7CoI436rbFwqGS9k3Mse/Wm5MyChWb3Dwvuic+6VNFmYaLhbvM7w== -->
 
 h7wbdX6KTqXpukbJbbAf2gDj36IqDtjx8U7CoI436rbFwqGS9k3Mse/Wm5MyChWb3Dwvuic+6VNFmYaLhbvM7w== 
 
 University.Instructor 
 
 name 
 
 Dr. Patel 
 
<!-- h7wbdX6KTqXpukbJbbAf2gDj36IqDtjx8U7CoI436rbFwqGS9k3Ms

The specification for offerings in a semester can be expressed in Factual. This format helps replicas to understand which facts their peers are interested in.

In [ ]:
offeringsForSemester.ToString()

(semester: University.Semester) {
    offering: University.Offering [
        offering->semester: University.Semester = semester
    ]
    course: University.Course [
        course = offering->course: University.Course
    ]
    instructor: University.Instructor [
        instructor = offering->instructor: University.Instructor
    ]
} => {
    CourseCode = course.code
    CourseName = course.name
    Days = offering.days
    Instructor = instructor.name
    Locations = {
        location: University.Offering.Location [
            location->offering: University.Offering = offering
            !E {
                next: University.Offering.Location [
                    next->prior: University.Offering.Location = location
                ]
            }
        ]
    } => {
        Building = location.building
        Room = location.room
    }
    Time = offering.time
}


## Application Components

The application is composed of client apps, functions, and services that produce and consume data. These components are distributed to different areas of the network in order to move them closer to the users that they serve, and isolate them from unwanted actors. The primary components are:

- **Student Portal**, where students register for courses and access their course history
- **Course Search Index**, which serves the student portal by providing access to the complete catalog of course offerings
- **Curriculum Provider Integration**, with which training organizations and partners manage course materials
- **Transcript Access**, where partnering universities obtain student course history and credentials

For each of these components, we can define a set of specifications that describe the data that they are interested in. These specifications control the distribution of data through the network.

### Student Portal

A student logs into the student portal to see their current and past course registrations. The presentation of this information might take the view of a schedule for the current semester. Or it might be presented as a transcript of past course completion. In either case, the specification for this information begins with the student and includes all registrations, offerings, and courses.

The schedule is produced based on a given student and semester. Find all registrations that have not been dropped. Then find the offerings of those registrations that are in the current semester. For each one, project information about the course and instructor.

In [ ]:
var currentSemesterSchedule = Given<Student, Semester>.Match((student, semester) =>
    from registration in student.Successors().OfType<Registration>(registration => registration.enrollment.application.student)
    where registration.Successors().No<Drop>(drop => drop.registration)
    from offering in registration.offering.Successors().OfType<Offering>(offering => offering)
    where offering.semester == semester
    from course in offering.course.Successors().OfType<Course>(course => course)
    from instructor in offering.instructor.Successors().OfType<Instructor>(instructor => instructor)
    select new
    {
        CourseCode = course.code,
        CourseName = course.name,
        Days = offering.days,
        Time = offering.time,
        Instructor = instructor.name,
        Locations =
            from location in offering.Locations
            select new
            {
                Building = location.building,
                Room = location.room
            }
    });
currentSemesterSchedule.ToString()

(student: University.Student, semester: University.Semester) {
    registration: University.Registration [
        registration->enrollment: University.Enrollment->application: University.Application->student: University.Student = student
        !E {
            drop: University.Drop [
                drop->registration: University.Registration = registration
            ]
        }
    ]
    offering: University.Offering [
        offering = registration->offering: University.Offering
        offering->semester: University.Semester = semester
    ]
    course: University.Course [
        course = offering->course: University.Course
    ]
    instructor: University.Instructor [
        instructor = offering->instructor: University.Instructor
    ]
} => {
    CourseCode = course.code
    CourseName = course.name
    Days = offering.days
    Instructor = instructor.name
    Locations = {
        location: University.Offering.Location [
            location->offering: University.Offering

The student portal will also display the history of course completions. One view will show all of the completed courses and their final grades.

In [ ]:
var completedCourses = Given<Student>.Match(student =>
    from complete in student.Successors().OfType<Complete>(complete => complete.registration.enrollment.application.student)
    from course in complete.registration.offering.course.Successors().OfType<Course>(course => course)
    select new
    {
        CourseCode = course.code,
        CourseName = course.name,
        Grade = complete.grade
    });
completedCourses.ToString()

(student: University.Student) {
    complete: University.Complete [
        complete->registration: University.Registration->enrollment: University.Enrollment->application: University.Application->student: University.Student = student
    ]
    course: University.Course [
        course = complete->registration: University.Registration->offering: University.Offering->course: University.Course
    ]
} => {
    CourseCode = course.code
    CourseName = course.name
    Grade = complete.grade
}


### Course Search Index

While students are using the registration portal, they need quick access to the current semester's course catalog. They might want to search by course code, description, or instructor. A search index is the best tool for this job.

To manage the search index, an application component will insert, update, and delete records as new facts become available. It will use a new fact to keep track of the records it is managing.

In [ ]:
[FactType("SearchIndex.Record")]
public record SearchIndexRecord(Offering offering, Guid recordId);

Renderer.RenderTypes(typeof(SearchIndexRecord))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- SearchIndex.Record -->
 
 SearchIndex.Record 
 
 SearchIndex.Record 
 
<!-- University.Offering -->
 
 University.Offering 
 
 University.Offering 
 
<!-- SearchIndex.Record->University.Offering -->
 
 SearchIndex.Record->University.Offering 
 
 
 offering 
 
<!-- University.Course -->
 
 University.Course 
 
 University.Course 
 
<!-- University.Offering->University.Course -->
 
 University.Offering->University.Course 
 
 
 course 
 
<!-- University.Semester -->
 
 University.Semester 
 
 University.Semester 
 
<!-- University.Offering->University.Semester -->
 
 University.Offering->University.Semester 
 
 
 semester 
 
<!-- University.Instructor -->
 
 University.Instructor 
 
 University.Instructor 
 
<!-- University.Offering->University.Instructor -->
 
 University.Offering->University.Instructor 
 
 
 instructor 
 
<!-- University.Organization -->
 
 University.Organization 
 
 University.Organization 
 
<!-- University.Course->University.Organization -->
 
 University.Course->University.Organization 
 
 
 organization 
 
<!-- University.Semester->University.Organization -->
 
 University.Semester->University.Organization 
 
 
 organization 
 
<!-- University.Instructor->University.Organization -->
 
 University.Instructor->University.Organization 
 
 
 organization

The indexer component uses a specification to look for offerings of a given semester that have not yet been indexed. When it finds them, it inserts a record into the index. Then it records the ID of that new record.

In [ ]:
class SearchRecord
{
    public string CourseCode { get; init; }
    public string CourseName { get; init; }
    public string Days { get; init; }
    public string Time { get; init; }
    public string Instructor { get; init; }
    public string Location { get; set; }
}
Dictionary<Guid, SearchRecord> index = new Dictionary<Guid, SearchRecord>();

var offeringsToIndex = Given<Semester>.Match(semester =>
    from offering in semester.Successors().OfType<Offering>(offering => offering.semester)
    where offering.Successors().No<OfferingDelete>(deleted => deleted.offering)
    where offering.Successors().No<SearchIndexRecord>(record => record.offering)
    select offering);
var indexInsertSubscription = j.Subscribe(offeringsToIndex, currentSemester, async offering =>
{
    // Create a record for the offering
    var recordId = Guid.NewGuid();
    index[recordId] = new SearchRecord
    {
        CourseCode = offering.course.code,
        CourseName = offering.course.name,
        Days = offering.days,
        Time = offering.time,
        Instructor = offering.instructor.name,
        Location = "TBA"
    };
    await j.Fact(new SearchIndexRecord(offering, recordId));
});

offeringsToIndex.ToString()

(semester: University.Semester) {
    offering: University.Offering [
        offering->semester: University.Semester = semester
        !E {
            deleted: University.Offering.Delete [
                deleted->offering: University.Offering = offering
            ]
        }
        !E {
            record: SearchIndex.Record [
                record->offering: University.Offering = offering
            ]
        }
    ]
} => offering


The indexer is subscribing to the specification. That causes it to hold a persistent connection to its upstream replicator. The replicator will send any new offers to the indexer for it to process immediately.

The indexer's specification filters out any offerings that have already been added to the index. This effectively pops the work off the queue once it has been completed.

We can see the resulting records in the search index.

In [ ]:
index.Values.AsTable()

index,CourseCode,CourseName,Days,Time,Instructor,Location
0,CS 101,Introduction to Computer Science,TTr,10:00,Dr. Patel,TBA
1,CS 401,Artificial Intelligence,WF,14:00,Dr. Smith,TBA
2,CS 401,Artificial Intelligence,TTr,16:00,Dr. Smith,TBA
3,CS 101,Introduction to Computer Science,TTr,13:00,Dr. Jones,TBA
4,CS 301,Software Engineering,WF,19:00,Dr. Jones,TBA
5,CS 101,Introduction to Computer Science,MF,14:00,Dr. Kim,TBA
6,CS 201,Data Structures and Algorithms,TTr,14:00,Dr. Smith,TBA
7,CS 601,Quantum Computing,MF,17:00,Dr. Singh,TBA
8,CS 201,Data Structures and Algorithms,TTr,10:00,Dr. Patel,TBA
9,CS 501,Machine Learning,MW,11:00,Dr. Lee,TBA


And we can see the facts keeping track of those records in the replica.

In [ ]:
var recordsCreatedInSemester = Given<Semester>.Match(semester =>
    from record in semester.Successors().OfType<SearchIndexRecord>(record => record.offering.semester)
    select record);
var currentRecords = await j.Query(recordsCreatedInSemester, currentSemester);
j.RenderFacts(currentRecords[0])

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== -->
 
 6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== 
 
 University.Organization 
 
 identifier 
 
 6003 
 
<!-- YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg== -->
 
 YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg== 
 
 University.Course 
 
 code 
 
 CS 101 
 
 name 
 
 Introduction to Comp... 
 
<!-- YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== -->
 
 YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== 
 
 
 organization 
 
<!-- cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg== -->
 
 cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg== 
 
 University.Semester 
 
 year 
 
 2022 
 
 term 
 
 Summer 
 
<!-- cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== -->
 
 cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== 
 
 
 organization 
 
<!-- h7wbdX6KTqXpukbJbbAf2gDj36IqDtjx8U7CoI436rbFwqGS9k3Mse/Wm5MyChWb3Dwvuic+6VNFmYaLhbvM7w== -->
 
 h7wbdX6KTqXpukbJbbAf2gDj36IqDtjx8U7CoI436rbFwqGS9k3Mse/Wm5MyChWb3Dwvuic+6VNFmYaLhbvM7w== 
 
 University.Instructor 
 
 name 
 
 Dr. Patel 
 
<!-- h7wbdX6KTqXpukbJbbAf2gDj36IqDtjx8U7CoI436rbFwqGS9k3Mse/Wm5MyChWb3Dwvuic+6VNFmYaLhbvM7w==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== -->
 
 h7wbdX6KTqXpukbJbbAf2gDj36IqDtjx8U7CoI436rbFwqGS9k3Mse/Wm5MyChWb3Dwvuic+6VNFmYaLhbvM7w==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== 
 
 
 organization 
 
<!-- gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ== -->
 
 gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ== 
 
 University.Offering 
 
 days 
 
 TTr 
 
 time 
 
 10:00 
 
<!-- gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ==->YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg== -->
 
 gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ==->YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg== 
 
 
 course 
 
<!-- gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ==->cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg== -->
 
 gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ==->cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg== 
 
 
 semester 
 
<!-- gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ==->h7wbdX6KTqXpukbJbbAf2gDj36IqDtjx8U7CoI436rbFwqGS9k3Mse/Wm5MyChWb3Dwvuic+6VNFmYaLhbvM7w== -->
 
 gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ==->h7wbdX6KTqXpukbJbbAf2gDj36IqDtjx8U7CoI436rbFwqGS9k3Mse/Wm5MyChWb3Dwvuic+6VNFmYaLhbvM7w== 
 
 
 instructor 
 
<!-- R7e3Bev17efO/zbVZjo4sQpoqd0qdpRH3Jhwi4na+LExD0QhsBHKvDiZudBxYMIEri2C/lciUXNIRzOWzr1gaQ== -->
 
 R7e3Bev17efO/zbVZjo4sQpoqd0qdpRH3Jhwi4na+LExD0QhsB

### Index Record Update

The search index does not yet have the offering location. The location might change at any time, and so the offering location is captured as a separate fact. The indexer will use another fact type and specification to keep track of those changes and their updates.

In [ ]:
[FactType("SearchIndex.Record.LocationUpdate")]
public record SearchIndexRecordLocationUpdate(SearchIndexRecord record, OfferingLocation location);

Renderer.RenderTypes(typeof(SearchIndexRecordLocationUpdate))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- SearchIndex.Record.LocationUpdate -->
 
 SearchIndex.Record.LocationUpdate 
 
 SearchIndex.Record.LocationUpdate 
 
<!-- SearchIndex.Record -->
 
 SearchIndex.Record 
 
 SearchIndex.Record 
 
<!-- SearchIndex.Record.LocationUpdate->SearchIndex.Record -->
 
 SearchIndex.Record.LocationUpdate->SearchIndex.Record 
 
 
 record 
 
<!-- University.Offering.Location -->
 
 University.Offering.Location 
 
 University.Offering.Location 
 
<!-- SearchIndex.Record.LocationUpdate->University.Offering.Location -->
 
 SearchIndex.Record.LocationUpdate->University.Offering.Location 
 
 
 location 
 
<!-- University.Offering -->
 
 University.Offering 
 
 University.Offering 
 
<!-- SearchIndex.Record->University.Offering -->
 
 SearchIndex.Record->University.Offering 
 
 
 offering 
 
<!-- University.Offering.Location->University.Offering.Location -->
 
 University.Offering.Location->University.Offering.Location 
 
 
 prior 
 
<!-- University.Offering.Location->University.Offering -->
 
 University.Offering.Location->University.Offering 
 
 
 offering 
 
<!-- University.Course -->
 
 University.Course 
 
 University.Course 
 
<!-- University.Offering->University.Course -->
 
 University.Offering->University.Course 
 
 
 course 
 
<!-- University.Semester -->
 
 University.Semester 
 
 University.Semester 
 
<!-- University.Offering->University.Semester -->
 
 University.Offering->University.Semester 
 
 
 semester 
 
<!-- University.Instructor -->
 
 University.Instructor 
 
 University.Instructor 
 
<!-- University.Offering->University.Instructor -->
 
 University.Offering->University.Instructor 
 
 
 instructor 
 
<!-- University.Organization -->
 
 University.Organization 
 
 University.Organization 
 
<!-- University.Course->University.Organization -->
 
 University.Course->University.Organization 
 
 
 organization 
 
<!-- University.Semester->University.Organization -->
 
 University.Semester->University.Organization 
 
 
 organization 
 
<!-- University.Instructor->University.Organization -->
 
 University.Instructor->University.Organization 
 
 
 organization

To find the records that need to be updated, the indexer looks for offerings that have search index records as well as locations, but no corresponding update.

In [ ]:
var recordsToUpdate = Given<Semester>.Match(semester =>
    from offering in semester.Successors().OfType<Offering>(offering => offering.semester)
    from record in offering.Successors().OfType<SearchIndexRecord>(record => record.offering)
    from location in offering.Locations
    where !(
        from update in record.Successors().OfType<SearchIndexRecordLocationUpdate>(update => update.record)
        where update.location == location
        select update
    ).Any()
    select new { record, location }
);
var indexUpdateSubscription = j.Subscribe(recordsToUpdate, currentSemester, async work =>
{
    var record = work.record;
    var location = work.location;
    index[record.recordId].Location = location.building + " " + location.room;
    await j.Fact(new SearchIndexRecordLocationUpdate(record, location));
});

recordsToUpdate.ToString()

(semester: University.Semester) {
    offering: University.Offering [
        offering->semester: University.Semester = semester
    ]
    record: SearchIndex.Record [
        record->offering: University.Offering = offering
    ]
    location: University.Offering.Location [
        location->offering: University.Offering = offering
        !E {
            next: University.Offering.Location [
                next->prior: University.Offering.Location = location
            ]
        }
        !E {
            update: SearchIndex.Record.LocationUpdate [
                update->record: SearchIndex.Record = record
                update->location: University.Offering.Location = location
            ]
        }
    ]
} => {
    location = location
    record = record
}


The subscription finds all rooms and updates the search records.

In [ ]:
index.Values.AsTable()

index,CourseCode,CourseName,Days,Time,Instructor,Location
0,CS 101,Introduction to Computer Science,TTr,10:00,Dr. Patel,Building A 101
1,CS 401,Artificial Intelligence,WF,14:00,Dr. Smith,Building D 103
2,CS 401,Artificial Intelligence,TTr,16:00,Dr. Smith,Building C 101
3,CS 101,Introduction to Computer Science,TTr,13:00,Dr. Jones,Building B 104
4,CS 301,Software Engineering,WF,19:00,Dr. Jones,Building B 102
5,CS 101,Introduction to Computer Science,MF,14:00,Dr. Kim,Building A 102
6,CS 201,Data Structures and Algorithms,TTr,14:00,Dr. Smith,Building A 104
7,CS 601,Quantum Computing,MF,17:00,Dr. Singh,Building B 104
8,CS 201,Data Structures and Algorithms,TTr,10:00,Dr. Patel,Building B 104
9,CS 501,Machine Learning,MW,11:00,Dr. Lee,Building B 101


If an offering changes rooms, then the search index will be updated in response. Here we will relocate one of the offerings in the current semester.

In [ ]:
var offeringAndLocationInSemester = Given<Semester>.Match(semester =>
    from offering in semester.Successors().OfType<Offering>(offering => offering.semester)
    from location in offering.Locations
    select new { offering, location }
);
var offeringsAndLocations = await j.Query(offeringAndLocationInSemester, currentSemester);
await j.Fact(new OfferingLocation(offeringsAndLocations[0].offering, "Building E", "105", [offeringsAndLocations[0].location]));

$"Relocated {offeringsAndLocations[0].offering.course.code} by {offeringsAndLocations[0].offering.instructor.name} to Building E 105"

Relocated CS 101 by Dr. Patel to Building E 105

### Index Record Deletion

If an offering is deleted, the indexer should remove the record from the search index. When it removes them, it records that action as a new fact.

In [ ]:
[FactType("SearchIndex.Record.Delete")]
public record SearchIndexRecordDelete(SearchIndexRecord record, DateTime deletedAt);

Renderer.RenderTypes(typeof(SearchIndexRecordDelete))

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- SearchIndex.Record.Delete -->
 
 SearchIndex.Record.Delete 
 
 SearchIndex.Record.Delete 
 
<!-- SearchIndex.Record -->
 
 SearchIndex.Record 
 
 SearchIndex.Record 
 
<!-- SearchIndex.Record.Delete->SearchIndex.Record -->
 
 SearchIndex.Record.Delete->SearchIndex.Record 
 
 
 record 
 
<!-- University.Offering -->
 
 University.Offering 
 
 University.Offering 
 
<!-- SearchIndex.Record->University.Offering -->
 
 SearchIndex.Record->University.Offering 
 
 
 offering 
 
<!-- University.Course -->
 
 University.Course 
 
 University.Course 
 
<!-- University.Offering->University.Course -->
 
 University.Offering->University.Course 
 
 
 course 
 
<!-- University.Semester -->
 
 University.Semester 
 
 University.Semester 
 
<!-- University.Offering->University.Semester -->
 
 University.Offering->University.Semester 
 
 
 semester 
 
<!-- University.Instructor -->
 
 University.Instructor 
 
 University.Instructor 
 
<!-- University.Offering->University.Instructor -->
 
 University.Offering->University.Instructor 
 
 
 instructor 
 
<!-- University.Organization -->
 
 University.Organization 
 
 University.Organization 
 
<!-- University.Course->University.Organization -->
 
 University.Course->University.Organization 
 
 
 organization 
 
<!-- University.Semester->University.Organization -->
 
 University.Semester->University.Organization 
 
 
 organization 
 
<!-- University.Instructor->University.Organization -->
 
 University.Instructor->University.Organization 
 
 
 organization

To find work to do, it creates another subscription. It looks for records for offerings that have been deleted.

In [ ]:
var recordsToDeleteInSemester = Given<Semester>.Match(semester =>
    from record in semester.Successors().OfType<SearchIndexRecord>(record => record.offering.semester)
    where record.Successors().No<SearchIndexRecordDelete>(recordDelete => recordDelete.record)
    where record.offering.Successors().Any<OfferingDelete>(offeringDelete => offeringDelete.offering)
    select record);
var indexDeleteSubscription = j.Subscribe(recordsToDeleteInSemester, currentSemester, async record =>
{
    // Delete the record from the index
    index.Remove(record.recordId);
    await j.Fact(new SearchIndexRecordDelete(record, DateTime.UtcNow));
});

recordsToDeleteInSemester.ToString()

(semester: University.Semester) {
    record: SearchIndex.Record [
        record->offering: University.Offering->semester: University.Semester = semester
        !E {
            recordDelete: SearchIndex.Record.Delete [
                recordDelete->record: SearchIndex.Record = record
            ]
        }
        E {
            offeringDelete: University.Offering.Delete [
                offeringDelete->offering: University.Offering = record->offering: University.Offering
            ]
        }
    ]
} => record


Now that that subscription is running, we will delete an offering.

In [ ]:
var offeringsInSemester = Given<Semester>.Match(semester =>
    from offering in semester.Successors().OfType<Offering>(o => o.semester)
    select offering);
var currentOfferings = await j.Query(offeringsInSemester, currentSemester);
await j.Fact(new OfferingDelete(currentOfferings[0], DateTime.UtcNow));

That fact triggers the indexer subscription and deletes the record. We can see that there is one fewer record in the index.

In [ ]:
index.Values.AsTable()

index,CourseCode,CourseName,Days,Time,Instructor,Location
0,CS 401,Artificial Intelligence,WF,14:00,Dr. Smith,Building D 103
1,CS 401,Artificial Intelligence,TTr,16:00,Dr. Smith,Building C 101
2,CS 101,Introduction to Computer Science,TTr,13:00,Dr. Jones,Building B 104
3,CS 301,Software Engineering,WF,19:00,Dr. Jones,Building B 102
4,CS 101,Introduction to Computer Science,MF,14:00,Dr. Kim,Building A 102
5,CS 201,Data Structures and Algorithms,TTr,14:00,Dr. Smith,Building A 104
6,CS 601,Quantum Computing,MF,17:00,Dr. Singh,Building B 104
7,CS 201,Data Structures and Algorithms,TTr,10:00,Dr. Patel,Building B 104
8,CS 501,Machine Learning,MW,11:00,Dr. Lee,Building B 101
9,CS 601,Quantum Computing,MW,14:00,Dr. Kim,Building C 102


And we can see the deletion fact.

In [ ]:
var recordsDeletedInSemester = Given<Semester>.Match(semester =>
    from delete in semester.Successors().OfType<SearchIndexRecordDelete>(delete => delete.record.offering.semester)
    from offeringDelete in delete.record.offering.Successors().OfType<OfferingDelete>(delete2 => delete2.offering)
    select new
    {
        delete,
        offeringDelete
    });
var currentDeletes = await j.Query(recordsDeletedInSemester, currentSemester);
j.RenderFacts(currentDeletes[0])

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.2.1 (20241206.2353)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== -->
 
 6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== 
 
 University.Organization 
 
 identifier 
 
 6003 
 
<!-- YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg== -->
 
 YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg== 
 
 University.Course 
 
 code 
 
 CS 101 
 
 name 
 
 Introduction to Comp... 
 
<!-- YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== -->
 
 YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== 
 
 
 organization 
 
<!-- cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg== -->
 
 cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg== 
 
 University.Semester 
 
 year 
 
 2022 
 
 term 
 
 Summer 
 
<!-- cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== -->
 
 cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== 
 
 
 organization 
 
<!-- h7wbdX6KTqXpukbJbbAf2gDj36IqDtjx8U7CoI436rbFwqGS9k3Mse/Wm5MyChWb3Dwvuic+6VNFmYaLhbvM7w== -->
 
 h7wbdX6KTqXpukbJbbAf2gDj36IqDtjx8U7CoI436rbFwqGS9k3Mse/Wm5MyChWb3Dwvuic+6VNFmYaLhbvM7w== 
 
 University.Instructor 
 
 name 
 
 Dr. Patel 
 
<!-- h7wbdX6KTqXpukbJbbAf2gDj36IqDtjx8U7CoI436rbFwqGS9k3Mse/Wm5MyChWb3Dwvuic+6VNFmYaLhbvM7w==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== -->
 
 h7wbdX6KTqXpukbJbbAf2gDj36IqDtjx8U7CoI436rbFwqGS9k3Mse/Wm5MyChWb3Dwvuic+6VNFmYaLhbvM7w==->6DQQ2IAaTZER2l722taIuu2pGXnr3P7cGFjxkjXxe4yxcGTDU8LD060o7rBTGu1yGjFiI+YOJaEcdDoBVUrR4Q== 
 
 
 organization 
 
<!-- gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ== -->
 
 gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ== 
 
 University.Offering 
 
 days 
 
 TTr 
 
 time 
 
 10:00 
 
<!-- gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ==->YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg== -->
 
 gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ==->YaUXufuHJF06p/BRSvMYG7pZKzX0iGBoPugG+99Qf2Bd6xlinvT7YMzZZSmrh/Hov249LxHynec0TdjPSwxtRg== 
 
 
 course 
 
<!-- gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ==->cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg== -->
 
 gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ==->cUBtum7heFMOg8RyqnI8PhbGiP2wQ6x2aOeGv/soI3fktRsSUls+CPJ4X8oEPn1hFllrvU6JeM2lSrAroEh2vg== 
 
 
 semester 
 
<!-- gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ==->h7wbdX6KTqXpukbJbbAf2gDj36IqDtjx8U7CoI436rbFwqGS9k3Mse/Wm5MyChWb3Dwvuic+6VNFmYaLhbvM7w== -->
 
 gHH5esVUUpLHJqMpUrHVspu2v9qAl2wPIQqP2vIu9R+KnDjaqctZHmo2oqRmxdu/Bwf1IFxIm8IGWiD1TXiWfQ==->h7wbdX6KTqXpukbJbbAf2gDj36IqDtjx8U7CoI436rbFwqGS9k3Mse/Wm5MyChWb3Dwvuic+6VNFmYaLhbvM7w== 
 
 
 instructor 
 
<!-- R7e3Bev17efO/zbVZjo4sQpoqd0qdpRH3Jhwi4na+LExD0QhsBHKvDiZudBxYMIEri2C/lciUXNIRzOWzr1gaQ== -->
 
 R7e3Bev17efO/zbVZjo4sQpoqd0qdpRH3Jhwi4na+LExD0QhsB

### When Are Indexes Updated?

The runtime determines when to update the index. It does so based on the specification. When it learns about a new fact that affects the subscription, it calls the application.

When a new fact arrives, the runtime executes a set of inverse specifications. If one returns a result that matches an active subscription, then the runtime knows that subscription was affected. It also knows exactly how that subscription was affected.

Here are the inverses of the offerings to index.

In [ ]:
string DescribeInverse(Jinaga.Pipelines.Inverse inverse)
{
    string verb = inverse.Operation == Jinaga.Pipelines.InverseOperation.Add ? "add" : "remove";
    string preposition = inverse.Operation == Jinaga.Pipelines.InverseOperation.Add ? "to" : "from";
    return $"When {inverse.InverseSpecification}Then {verb} ({inverse.ResultSubset}) {preposition} ({inverse.ParentSubset})\n";
}

string.Join("\n", offeringsToIndex.ComputeInverses().Select(DescribeInverse))

When (offering: University.Offering [
    !E {
        deleted: University.Offering.Delete [
            deleted->offering: University.Offering = offering
        ]
    }
    !E {
        record: SearchIndex.Record [
            record->offering: University.Offering = offering
        ]
    }
]) {
    semester: University.Semester [
        semester = offering->semester: University.Semester
    ]
} => offering
Then add (semester, offering) to (semester)

When (deleted: University.Offering.Delete) {
    offering: University.Offering [
        offering = deleted->offering: University.Offering
        !E {
            record: SearchIndex.Record [
                record->offering: University.Offering = offering
            ]
        }
    ]
    semester: University.Semester [
        semester = offering->semester: University.Semester
    ]
} => offering
Then remove (semester, offering) from (semester)

When (record: SearchIndex.Record) {
    offering: University.Offering [
        offering

In [ ]:
string.Join("\n", recordsToUpdate.ComputeInverses().Select(DescribeInverse))

When (record: SearchIndex.Record) {
    offering: University.Offering [
        offering = record->offering: University.Offering
    ]
    semester: University.Semester [
        semester = offering->semester: University.Semester
    ]
    location: University.Offering.Location [
        location->offering: University.Offering = offering
        !E {
            next: University.Offering.Location [
                next->prior: University.Offering.Location = location
            ]
        }
        !E {
            update: SearchIndex.Record.LocationUpdate [
                update->record: SearchIndex.Record = record
                update->location: University.Offering.Location = location
            ]
        }
    ]
} => {
    location = location
    record = record
}
Then add (semester, offering, record, location) to (semester)

When (location: University.Offering.Location [
    !E {
        next: University.Offering.Location [
            next->prior: University.Offering.Location =